In [ ]:
!pip install nltk faiss-cpu llama-index-core==0.11.17 llama-index-vector-stores-faiss langchain-huggingface pinecone tran

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

df = pd.read_csv('/content/message_data.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# **Cleaning and Preprocessing**

In [ ]:
# Function to clean text
def clean_text(text):
  # Replace NaN values with an empty string
  df['Subject'] = df['Subject'].fillna('')
  df['Body'] = df['Body'].fillna('')

  if isinstance(text, str):
    # Remove links (URLs)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

     # Convert to lowercase
    text = text.lower()

  return text

# Function to remove stop words
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Subject  87 non-null     object
 1   Date     87 non-null     object
 2   Body     76 non-null     object
dtypes: object(3)
memory usage: 2.2+ KB


In [ ]:
# Clean and preprocess subject and body columns
df['cleaned_subject'] = df['Subject'].apply(clean_text).apply(remove_stopwords)
df['cleaned_body'] = df['Body'].apply(clean_text).apply(remove_stopwords)

# Combine cleaned subject and body for embedding
combined_texts = [
    f"Subject: {row['cleaned_subject']} Body: {row['cleaned_body']}"
    for index, row in df.iterrows()
]
print(combined_texts[0])

Subject: become nextjs pro Body: hey nextjs become wellknown ability build fast reliable full stack apps good reason developer experience top notch wide range features help easily build performancefirst apps able focus unique challenges app nextjs 15 react 19 coming soon important ever make sure full understanding nextjs works newer react features help build amazing experiences new course launched help build full stack invoice app using nextjs 15 learn intricacies nextjs 15 react 19 well also learn design responsive components using tailwind shadcnui add authentication social login organization support mfa clerk create manage databases relationships accross tables xata query wrangle data postgres server xata drizzle orm process payments invoices using stripe build custom email templates react react email send transaction emails resend deploy app vercel looooots important concepts inbetween best part course available free thanks course partners xata clerk making possible ready get start

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-base-en-v1.5"

# Generate embeddings for combined texts
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name)
embeddings = hf_embeddings.embed_documents(combined_texts)

dimention = len(embeddings[0])
print(dimention)  # Dimensionality of embedding space

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

768


In [ ]:
!pip install langchain-pinecone langchain_community pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.10.10
    Uninstalling aiohttp-3.10.10:
      Successfully uninstalled aiohttp-3.10.10


In [ ]:
import os

os.environ['PINECONE_API_KEY'] = '88c2edde-2d71-480a-aeff-62c7061bd7f2'
os.environ['PINECONE_API_ENV'] = '<YOUR_PINECONE_ENVIRONMENT>'

In [ ]:
from pinecone import Pinecone as PineconeClient, ServerlessSpec

pc = PineconeClient(api_key="88c2edde-2d71-480a-aeff-62c7061bd7f2")

index_name = "example-index"
pinecone_index = pc.Index(index_name)

# run only once

# dimentions = len(embeddings[0])
# pc.create_index(
#   name="example-index",
#   dimension=dimentions,
#   metric="cosine",
#   spec=ServerlessSpec(
#     cloud="aws",
#     region="us-east-1"
#   )
# )

In [ ]:
import os
from langchain_pinecone import PineconeVectorStore

# examples
# texts = ["Tonight, I call on the Senate to: Pass the Freedom to Vote Act.", "ne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.", "One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence."]

vectorstore_from_texts = PineconeVectorStore.from_texts(
    texts,
    index_name=index_name,
    embedding=hf_embeddings
)


print("Pinecone VectorStoreIndex successfully initialized using Hugging Face embeddings!")

NameError: name 'texts' is not defined

In [ ]:
vectorstore.add_texts(combined_texts)

NameError: name 'vectorstore' is not defined

In [ ]:
!pip install sentence_transformers langchain-huggingface

In [ ]:
# Initialize the vectorstore if already exists
from langchain_pinecone import PineconeVectorStore
# from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings

hf_embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

vector_store = PineconeVectorStore(index_name=index_name, embedding=hf_embed_model)

print("Vector Store initialized!")

Vector Store initialized!


In [ ]:
query = "What is dropbox basic basic account size?"
vector_store.similarity_search(query, k=2)

[Document(id='2f38abae-5efa-4ca2-82a8-eb91477958e0', metadata={}, page_content='Subject: sharings easy dropbox Body: everyone gets 500 mb free space sharings easy dropbox invite friends dropbox get 16 gb bonus storage referring friends get bonus storage well complete signup process need dropbox need space everybody wins send invite ____________________________________________________ dropbox inc po box 77767 san francisco ca 94107 view privacy policy2 unsubscribe3 1 2 3'),
 Document(id='f6efdb4a-127a-4113-8401-5f6fc116e5ab', metadata={}, page_content='Subject: want Body: learn make life easier dropbox else dropbox probably signed dropbox store share couple files get started youll find helpful cloud storage learn plus family help get dropbox learn dropbox plus level file storage sharing limits builtin tools help organize digital life save share photos videos sensitive documents 2000 gb secure cloud storage access everything need 247 synced across devices dropbox passwords easily sign we

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Testing llms
text = "Who is Queen Elizabeth?"
inputs = tokenizer(text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Who is Queen Elizabeth?
Answer: Queen Elizabeth is the Queen of England.

Question: What is the title of the song?
Answer: The title of the song is "Queen Elizabeth".

Question: Who wrote the song?
Answer: The song


In [ ]:
def generate_email_reply(query):
    # Perform similarity search in the vector store
    results = vector_store.similarity_search(query, k=2)

    # Extract relevant content from the search results
    relevant_texts = [result.page_content for result in results]

    # Construct a prompt for the LLM
    prompt = f"Based on the following information, reply to this email:\n\n{query}\n\nRelevant information:\n" + "\n".join(relevant_texts)

    # Tokenize input and generate response
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(**inputs, max_new_tokens=50)

    # Decode and return the reply from the LLM's output
    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return reply

# Example usage
email_query = "How much free space dropbox provides for sharing?"
reply = generate_email_reply(email_query)
print(reply)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Based on the following information, reply to this email:

How much free space dropbox provides for sharing?

Relevant information:
Subject: sharings easy dropbox Body: everyone gets 500 mb free space sharings easy dropbox invite friends dropbox get 16 gb bonus storage referring friends get bonus storage well complete signup process need dropbox need space everybody wins send invite ____________________________________________________ dropbox inc po box 77767 san francisco ca 94107 view privacy policy2 unsubscribe3 1 2 3
Subject: want Body: learn make life easier dropbox else dropbox probably signed dropbox store share couple files get started youll find helpful cloud storage learn plus family help get dropbox learn dropbox plus level file storage sharing limits builtin tools help organize digital life save share photos videos sensitive documents 2000 gb secure cloud storage access everything need 247 synced across devices dropbox passwords easily sign websites apps devices check plus d

In [ ]:
df.head(5)

,Subject,Date,Body,cleaned_subject,cleaned_body
0,Become a Next.js Pro,2024-10-02 21:43:27+05:30,Hey there 👋\r\n\r\nNext.js has become well-kno...,become nextjs pro,hey nextjs become wellknown ability build fast...
1,Your Gamma PPTX is ready,2024-10-01 11:00:49+05:30,A message from Gamma ( https://click.pstmrk.it...,gamma pptx ready,message gamma 2024 gamma tech inc
2,Your Gamma PPTX is ready,2024-09-28 13:05:37+05:30,A message from Gamma ( https://click.pstmrk.it...,gamma pptx ready,message gamma 2024 gamma tech inc
3,🎁 Here’s your free Motion Graphics template,2024-04-19 05:51:23+05:30,View web version:\r\nhttps://t-info.mail.adobe...,heres free motion graphics template,view web version unfortunately email client ca...
4,Stand out with the new Adobe Express mobile app,2024-04-18 22:03:17+05:30,\r\n\r\nView web version:\r\nhttps://t-info.ma...,stand new adobe express mobile app,view web version unfortunately email client ca...


In [ ]:
df.describe()

,Subject,Date,Body,cleaned_subject,cleaned_body
count,87,87,87,87,87
unique,78,87,77,78,64
top,🎉 Your free words are here!,2024-10-02 21:43:27+05:30,,free words,
freq,3,1,11,3,11


In [ ]:
from transformers import pipeline

# Load a pre-trained LLM (e.g., T5 or GPT-2)
email_reply_model = pipeline("text-generation", model="gpt2")

def generate_reply(subject, body):
    input_text = f"Subject: {subject}\n\n{body}\n\nReply:"
    reply = email_reply_model(input_text, max_length=100, num_return_sequences=1)[0]['generated_text']
    return reply


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
generate_reply(df['cleaned_subject'][0] , df['cleaned_body'][1])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Subject: become nextjs pro\n\nmessage gamma 2024 gamma tech inc\n\nReply: we need to work from our own server to get some extra bytes.\n\nReply: why not make the nextjs web interface just before making nextjs\n\nReply: nextjs is awesome for now\n\nReply: my own server does not support a backend as I have already worked with several developers at this stage.\n\nReply: not sure if I should have asked them to make the nextjs'

In [ ]:
df['cleaned_subject'][0] , df['cleaned_body']

('become nextjs pro',
 0     hey nextjs become wellknown ability build fast...
 1                     message gamma 2024 gamma tech inc
 2                     message gamma 2024 gamma tech inc
 3     view web version unfortunately email client ca...
 4     view web version unfortunately email client ca...
                             ...                        
 82    hi shogun welcome google new mac next take 2 s...
 83    hi shogun welcome google new mac next take 2 s...
 84    fields justice await congratulations acceptanc...
 85    riot id createdfafgaming 4373 riot id used pla...
 86    riot account createdyour riot account created ...
 Name: cleaned_body, Length: 87, dtype: object)